# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/hinzlehome/codeup-data-science/clustering-project/')

from utils.imports import *
from utils.wrangle import *
from utils.model import *


TabError: inconsistent use of tabs and spaces in indentation (model.py, line 160)

---

# Wrangle

In [ ]:
X_train, y_train, X_validate, y_validate, X_test, y_test=sml_zillow(['logerror'])

In [ ]:
X_train.info()

---

# Cluster

In [ ]:
# cols=train.columns.tolist()

In [ ]:
cols=['calculatedfinishedsquarefeet','taxvaluedollarcnt']

In [ ]:
inertia(X_train,cols,1,6)

# kmeans object ; helps identify best k for cluster 
# "inertia" : sum of the squared distances from each point to it's assigned centroid

In [ ]:
# call cluster function to add 'cluster' col to df with each entries kmeans cluster
X_train,centroids=cluster(X_train,'calculatedfinishedsquarefeet','taxvaluedollarcnt',2)

In [ ]:
sns.relplot(data=X_train,x='calculatedfinishedsquarefeet',y='taxvaluedollarcnt',hue='cluster')

---

# Regression

### Baseline

Establish baseline to determine the validity of our various models. In this case median is used since outliers have not been addressed.

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
mses, y_train, pf=reg_zillow_train(X_train,y_train)

In [ ]:
def reg_zillow_train(X_train,y_train):

	y_train['yhat_base_prop']=y_train.logerror.mean()


	# evaluate: rmse
	mse_baseline = mean_squared_error(y_train.logerror, y_train.yhat_base_prop)

	model = LinearRegression().fit(X_train[['cluster']],y_train['logerror'])

	y_train['yhat_prop_ols'] = model.predict(X_train[['cluster']])

	mse_ols = mean_squared_error(y_train.logerror, y_train.yhat_prop_ols)

	print("MSE OLS sklearn: ","{:.2e}".format(mse_ols)) 

	if mse_ols-mse_baseline<0:
		print("y_hat_ols superior")
	else:
		print("yhat_baseline superior")

	lars = LassoLars(alpha=1.0)

	# create the model object

	lars.fit(X_train[['cluster']], y_train.logerror)

	# fit the model to our training data. We must specify the column in y_train, 
	# since we have converted it to a dataframe from a series!

	y_train['yhat_prop_lars'] = lars.predict(X_train[['cluster']])

	# predict train

	# evaluate: rmse
	mse_lars_train = mean_squared_error(y_train.logerror, y_train.yhat_prop_lars)


	print("MSE for Lasso + Lars\nTraining/In-Sample: ","{:.2e}".format(mse_lars_train))

	if mse_lars_train-mse_baseline<0:
		print("y_hat_lars superior")
	else:
		print("yhat_baseline superior")


	# create the model object
	glm = TweedieRegressor(power=1, alpha=0)

	# fit the model to our training data. We must specify the column in y_train, 
	# since we have converted it to a dataframe from a series! 
	glm.fit(X_train[['cluster']], y_train.logerror)

	# predict train
	y_train['yhat_prop_glm'] = glm.predict(X_train[['cluster']])

	# evaluate: mse
	mse_glm_train = mean_squared_error(y_train.logerror, y_train.yhat_prop_glm)

	print("MSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ","{:.2e}".format( mse_glm_train))

	if mse_glm_train-mse_baseline<0:
		print("y_hat superior")
	else:
		print("yhat_baseline superior")

	# make the polynomial features to get a new set of features
	pf = PolynomialFeatures(degree=2)

	# fit and transform X_train_scaled
	X_train_degree2 = pf.fit_transform(X_train[['cluster']])

	# create the model object
	lmp = LinearRegression(normalize=True)

	# fit the model to our training data. We must specify the column in y_train, 
	# since we have converted it to a dataframe from a series! 
	lmp.fit(X_train_degree2,y_train.logerror)

	# predict train
	y_train['yhat_prop_lmp'] = lmp.predict(X_train_degree2)

	# evaluate: rmse
	mse_lmp_train = mean_squared_error(y_train.logerror, y_train.yhat_prop_lmp)

	print("MSE for Polynomial Model, degrees=2\nTraining/In-Sample: ","{:.2e}".format( mse_lmp_train))

	if mse_glm_train-mse_baseline<0:
		print("y_hat superior")
	else:
		print("yhat_baseline superior")
		
	mses=pd.DataFrame([mse_baseline,
	mse_ols,
	mse_lars_train,
	mse_glm_train,
	mse_lmp_train],
	columns=['mse'],
	index= ['mse_baseline',
	'mse_ols',
	'mse_lars_train',
	'mse_glm_train',
	'mse_lmp_train'])
	print(mses)
	print(y_train)

	return mses, y_train, pf

### Validate

In [ ]:

	X_validate,centroids=cluster(X_validate,'calculatedfinishedsquarefeet','taxvaluedollarcnt',2)

	# transform X_validate_scaled 
	X_validate_degree2 = pf.transform(X_validate[['cluster']])


	y_validate['yhat_val_base'] = y_validate.logerror.mean()

	y_validate['yhat_val_ols'] = model.predict(X_validate[['cluster']])

	y_validate['yhat_val_lars'] = lars.predict(X_validate[['cluster']])

	y_validate['yhat_val_glm'] = glm.predict(X_validate[['cluster']])

	y_validate['yhat_val_lmp'] = lmp.predict(X_validate_degree2)



	mse_base_val = mean_squared_error(y_validate.logerror, y_validate.yhat_val_base)

	mse_ols_val = mean_squared_error(y_validate.logerror, y_validate.yhat_val_ols)

	mse_lars_val = mean_squared_error(y_validate.logerror, y_validate.yhat_val_lars)

	mse_glm_val = mean_squared_error(y_validate.logerror, y_validate.yhat_val_glm)

	mse_lmp_val = mean_squared_error(y_validate.logerror, y_validate.yhat_val_lmp)

	mse_val=[mse_base_val,mse_ols_val,mse_lars_val,mse_glm_val,mse_lmp_val]
	mse_val=pd.DataFrame(mse_val,index=['mse_base_val','mse_ols_val','mse_lars_val','mse_glm_val','mse_lmp_val' ],columns=['mse'])

	print(y_validate.mean())
	print(mse_val)

	return y_validate, mse_val

In [ ]:
y_validate, mse_val= reg_zillow_val(X_validate,y_validate,pf)

We can see that our median baseline is the most effective predictor in this case. The best model is the is Lasso Lars model. We will use this model for our test.

### Test

In [ ]:
X_test,centroids=cluster(X_test,'calculatedfinishedsquarefeet','taxvaluedollarcnt',2)

In [ ]:
X_test_degree2 = pf.transform(X_test[['cluster']])

In [ ]:
model = LinearRegression().fit(X_test[['cluster']],y_test['logerror'])

y_test['yhat_test_ols'] = model.predict(X_test[['cluster']])

mse_ols = mean_squared_error(y_test.logerror, y_test.yhat_test_ols)

print("MSE OLS sklearn: ","{:.2e}".format(mse_ols)) 

y_test['test_baseline'] = y_test['logerror'].median()

mse_test_baseline=mean_squared_error(y_test.logerror, y_test.test_baseline)

if mse_ols-mse_test_baseline<0:
	print("y_hat superior")
else:
	print("yhat_baseline superior")


---